#Explication: 
Ce code permet d'extraire les quatre parties (introduction, méthode, résultats, conclusion) **d'un résumé qui contient plus de quatre** sections. Pour cela nous utiliserons une liste d'étiquettes, pour savoir ou appartient chaque section. cette liste peut être augmentée, le code est disponible.

Attention le fichier d'entrée doit être un fichier json à la même structurre que celui dans le code ci-dessous (pour les nouvelles données, vous devez exécuter le code bash "code_extraction.sh" + "xml2json.py" , pour avoir les données dans ce format Json.

In [ ]:
import pandas as pd
df = pd.read_json('all-xml.json')

introduction_list = []
methods_list = []
results_list = []
conclusions_list = []
pmid_list = []
tab = []

### liste des étiquettes

In [ ]:

#TRIAL REGISTRATION, CLINICALTRIAL,TRIAL REGISTRATION INFORMATION,NETHERLANDS TRIAL REGISTER + TRIAL REGISTRATION + CLINICAL TRIAL REGISTRATION + Conclus\u00e3o + FUNDING + => à éliminer 
liste_introduction =['IMPORTANCE','BACKGROUND:','AIM:','PURPOSE/OBJECTIVES','PURPOSE', 'INTRODUCTION','BACKGROUND','OBJECTIVE','SUMMARY BACKGROUND DATA','AIM OF THE STUDY','BACKGROUND','Background','HYPOTHESIS','Objective','AIM','AIMS','BACKGROUND &amp','AIMS AND OBJECTIVES','HYPOTHESES','STUDY OBJECTIVE','PURPOSES/OBJECTIVES','RESEARCH QUESTION','WHAT IS ALREADY KNOWN ABOUT THIS SUBJECT','CONTEXT','CONTEXT AND OBJECTIVE','STUDY OBJECTIVES','OBJECTIVES','BACKGROUND AND OBJECTIVES','OBJECT','Background']
liste_methods = ['OUTCOME MEASURES','SUBJECT','METHOD:','MAIN OUTCOMES AND MEASURES','MAIN OUTCOME MEASURE','MAIN OUTCOME MEASURES','MEASURE','SAMPLE','METHODOLOGY','SETTING','OUTCOME MEASURE','MAIN OUTCOME MEASURE(S)','METHODS','PARTICIPANTS AND SETTING','RESEARCH DESIGN','DESIGN, PARTICIPANTS AND INTERVENTIONS','DESIGN AND SETTING','THEORETICAL FRAMEWORK','SCOPE','SETTING/LOCATION','TARGET POPULATION','SUBJECTS AND METHODS','DESIGN AND METHOD','CONTROL GROUP','POPULATION','PARTICIPANTS','MAIN RESEARCH VARIABLES','DESIGN, SETTING, AND PATIENTS','SETTINGS','MATERIALS AND METHODS','MATERIAL AND METHODS','OUTCOME MEASUREMENTS AND STATISTICAL ANALYSIS','STATISTICAL ANALYSIS USED','PATIENTS AND METHOD','DESIGN OF STUDY','DESIGN AND METHODS','SUBJECTS/PATIENTS','ANALYSIS','MEASUREMENTS','PARTICIPANTS/METHODS','PATIENT SUMMARY','METHOD','STATISTICAL ANALYSES','VARIABLES','INTERVENTIONS/METHODS','Subjects','Participants','PATIENTS AND METHODS', "DESIGN, SETTING, AND PARTICIPANTS",'DESIGN', 'SETTING', 'PARTICIPANTS AND INTERVENTIONS','PARTICIPANTS AND METHODS','Design','STUDY DESIGN','PROBANDS','DESIGN AND PARTICIPANTS','SETTING/PARTICIPANTS','STATISTICAL ANALYSES PERFORMED','SETTINGS/LOCATION','Settings','PATIENT CHARACTERISTICS','RESEARCH DESIGN','MEASUREMENT','SUBJECTS','INTERVENTION(S)','MAIN MEASURES','PLACE AND DURATION OF STUDY','INTERVENTION','DESIGN','Setting','Intervention','PARTICIPANTS/SETTING','PATIENTS','SETTING AND DESIGN','MAIN MEASURE','Methods','INTERVENTION/METHODS','DESIGN/SETTING','Measurements','METHOD','SETTING AND PARTICIPANTS','INTERVENTIONS','DESIGN AND SUBJECTS','ANALYSES','SUBJECTS/SETTING' ,'PRIMARY OUTCOME']
liste_resultats = ['RESULTS:',"RESULTS",'RESULTS','Results','MAIN RESULTS','RESULTS OF BASE-CASE ANALYSIS','MEASUREMENTS AND RESULTS','FINDINGS','KEY RESULTS','OUTCOMES','RESULTS','RESULTS AND LIMITATIONS','RESULTS OF SENSITIVITY ANALYSIS','SECONDARY OUTCOMES','PRINCIPAL FINDINGS', 'MEASUREMENTS AND MAIN RESULTS','MEASURES']
liste_conclusion = ['CONCLUSION:','CLINICAL RELEVANCE','PRACTICE IMPLICATIONS','CONCLUSIONS','CONCLUSION','IMPLICATIONS FOR NURSING','CONCLUSIONS AND IMPLICATIONS','RELEVANCE TO CLINICAL PRACTICE','CONCLUSIONS/IMPLICATIONS FOR PRACTICE','PRACTICE IMPLICATIONS','ELIGIBILITY','IMPLICATIONS FOR PRACTICE','IMPACT','INTERPRETATION','WHAT THIS STUDY ADDS','PERSPECTIVE','APPLICATIONS','DISCUSSION/CONCLUSIONS','IMPLICATION','IMPLICATIONS','IMPLICATIONS FOR CANCER SURVIVORS','Limitation','LIMITATIONS','Conclusion','PRACTICE IMPLICATION','SPONSORSHIP','CONCLUSION','APPLICATION','Discussion','CONCLUSION AND RELEVANCE','IMPLICATIONS FOR RESEARCH AND PRACTICE','APPLICATIONS/CONCLUSIONS','CONCLUSIONS AND RELEVANCE','KNOWLEDGE TRANSLATION','Conclusions','DISCUSSION','RELEVANCE','CLINICAL REHABILITATION IMPACT','IMPLICATIONS FOR NURSING PRACTICE']



In [ ]:
for i in range(df['pre'].shape[0]):
  if 'Abstract' in df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']:
    if type(df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText']) is list:# on verifie si c'est une liste  
      if len(df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'])>4:
        intro=''
        meth=''
        resul=''
        conc=''
        if all( type(abc) is dict for abc in df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText']):  #to check if all the  list contains only dictionaries
          for j in range(len(df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'])):
            label = ''
            psc = ''
            if type(df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'][j]) is dict: #si cest une dictionnaire parfois cest juste un str
              if '@Label' in df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'][j]:
                label = df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'][j]['@Label']
                if '#text' in  df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'][j]:              
                  psc = df['pre'][i]['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText'][j]['#text'] 
                  if psc:  
                    if label in liste_introduction:
                      intro = intro + str(psc)
                    if label in liste_methods:
                      meth = meth + str(psc)
                    if label in liste_resultats:
                      resul = resul + str(psc)
                    if label in liste_conclusion:
                      conc = conc + str(psc)
          
            
          if intro and meth and resul and conc:
            introduction_list.append(intro) 
            methods_list.append(meth)
            results_list.append(resul)
            conclusions_list.append(conc)

            pmid = df['pre'][i]['PubmedArticle']['MedlineCitation']['PMID']['#text']
            pmid_list.append(pmid)
          else:
            tab.append(pmid)

In [ ]:
pmid_df = pd.DataFrame(pmid_list,columns=['pmid'])

In [ ]:
intro_df = pd.DataFrame(introduction_list,columns=['introduction'])

In [ ]:
metho_df = pd.DataFrame(methods_list,columns=['methods'])

In [ ]:
resul_df = pd.DataFrame(results_list,columns=['resultats'])

In [ ]:
conc_df = pd.DataFrame(conclusions_list,columns=['conclusion'])

In [ ]:
result = pd.concat([pmid_df, intro_df, metho_df, resul_df, conc_df], axis=1, sort=False)

In [ ]:
result.to_csv('Abstract_devided_for_long_text.csv')